In [ ]:
import pandas as pd
import os, time
import numpy as np
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#ResNet50
data_dir = "data/north_american_predators/greyscale_downsampled_split/train"
dataset_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/greyscale_downsampled_split/val"
dataset_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/greyscale_downsampled_split/test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)
print(dataset_train.class_names)

Found 13823 files belonging to 8 classes.
Found 3454 files belonging to 8 classes.
Found 4317 files belonging to 8 classes.
['black_bear', 'brown_bear', 'cougar', 'coyote', 'human', 'non_human', 'polar_bear', 'wolf']


In [ ]:
def evaluate_model_tflite(interpreter, dataset):
    #List of dictionaries
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    correct = 0
    total = 0

    for images, labels in dataset:
        input_type = input_details[0]['dtype']
        # Convert the batch to the correct type
        images = images.numpy().astype(input_type)
        labels = labels.numpy()
        
        for i in range(images.shape[0]):
            test_image = np.expand_dims(images[i], axis=0)
            #From Tensorflow API Docs
            interpreter.set_tensor(input_details[0]['index'], test_image)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            #Gets the most prominent class
            pred = np.argmax(output)
            if pred == labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [4]:
base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
early_stop = EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

start_time = time.time()
model.fit(dataset_train, epochs=100, validation_data=dataset_val, callbacks=[early_stop])
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Epoch 1/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 182s 831ms/step - loss: 0.2347 - sparse_categorical_accuracy: 0.9285 - val_loss: 0.4704 - val_sparse_categorical_accuracy: 0.8500
Epoch 2/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 176s 816ms/step - loss: 0.0696 - sparse_categorical_accuracy: 0.9783 - val_loss: 0.4477 - val_sparse_categorical_accuracy: 0.8691
Epoch 3/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 180s 833ms/step - loss: 0.0499 - sparse_categorical_accuracy: 0.9843 - val_loss: 0.4866 - val_sparse_categorical_accuracy: 0.8683
Epoch 4/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 174s 807ms/step - loss: 0.0403 - sparse_categorical_accuracy: 0.9871 - val_loss: 0.4590 - val_sparse_categorical_accuracy: 0.8770
Epoch 5/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 174s 805ms/step - loss: 0.0331 - sparse_categorical_accuracy: 0.9896 - val_loss: 0.5323 - val_sparse_categorical_accuracy: 0.8668
Epoch 6/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 170s 789ms/step - loss: 0.0266 - sparse_categorical_accuracy: 0.9925 - val_loss: 0.5097 - val_sparse_cat

In [6]:
start_time = time.time()
test_loss, test_acc = model.evaluate(dataset_test, verbose=0)
end_time = time.time()
training_duration = end_time - start_time
print(f"Evaluation time: {training_duration:.2f} seconds")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Evaluation time: 43.10 seconds
Test Loss: 0.2540
Test Accuracy: 91.55%


In [7]:
model.save('model_ResNet50_grey.keras')
print("Baseline model in Mb:", os.path.getsize('model_ResNet50_grey.keras') / float(2**20))

Baseline model in Mb: 90.78868198394775


In [8]:
loaded_model = tf.keras.models.load_model("model_ResNet50_grey.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_ResNet50_quantized_grey.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp2xj12hj1\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp2xj12hj1\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmp2xj12hj1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  3201587513488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587511952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587511568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587515216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587515024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587514448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587514832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587513104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587511760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3201587516368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  320158

In [9]:
loaded_model = tf.keras.models.load_model("model_ResNet50_grey.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_ResNet50_grey.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpd3i638ad\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpd3i638ad\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmpd3i638ad'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  3201282995024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3202807120336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3202807120144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3203209461008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3202807119952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3202807121680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3203209461392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3203209460624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3203209461200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3203209459472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  320320

In [ ]:
#ResNet50 trained on greyscale images
interpreter = tf.lite.Interpreter(model_path="model_ResNet50_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_ResNet50_grey.tflite') / float(2**20))

c:\Graduate_School\Fall_2025\ECE528\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


TFLite accuracy: 0.9154505443595089
TFlite model in Mb: 89.66934967041016


In [ ]:
#ResNet50 quantized trained on greyscale images
interpreter = tf.lite.Interpreter(model_path="model_ResNet50_quantized_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_ResNet50_quantized_grey.tflite') / float(2**20))

TFLite accuracy: 0.9152189020152884
TFlite model in Mb: 22.82269287109375
